## 中文语言处理

和拉丁语系不同，亚洲语言是不用空格分开每个有意义的词的。而当我们进行自然语言处理的时候，大部分情况下，词汇是我们对句子和文章理解的基础，因此需要一个工具去把完整的文本中分解成粒度更细的词。

## 关键词提取

### 基于 TF-IDF 算法的关键词抽取

``` 
import jieba.analyse
 × jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
        × sentence 为待提取的文本
        × topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
        × withWeight 为是否一并返回关键词权重值，默认值为 False
        × allowPOS 仅包括指定词性的词，默认值为空，即不筛选
```

In [2]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)
print("  ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.547 seconds.
Prefix dict has been built succesfully.


用户  2016  互联网  手机  平台  人工智能  百度  2017  智能  技术  数据  360  服务  直播  产品  企业  安全  视频  移动  应用  网络  行业  游戏  机器人  电商  内容  中国  领域  通过  发展


In [5]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)
print("  ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))

航母  训练  海军  中国  官兵  部队  编队  作战  10  任务  美国  导弹  能力  20  2016  军事  无人机  装备  进行  记者  我们  军队  安全  保障  12  战略  军人  日本  南海  战机


### 基于 TextRank 算法的关键词抽取

> jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。

> jieba.analyse.TextRank() 新建自定义 TextRank 实例

算法论文：[<i class="fa fa-external-link-square" aria-hidden="true"> TextRank: Bringing Order into Texts</i>](http://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

基本思想:

- 将待抽取关键词的文本进行分词
- 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
- 计算图中节点的PageRank，注意是无向带权图

In [8]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)

print("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))
print("---------------------我是分割线----------------")
print("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

中国  海军  训练  美国  部队  进行  官兵  航母  作战  任务  能力  军事  发展  工作  国家  问题  建设  导弹  编队  记者
---------------------我是分割线----------------
中国  海军  美国  部队  官兵  航母  军事  国家  任务  能力  导弹  技术  问题  日本  军队  编队  装备  系统  记者  战略


## LDA主题模型


下面用LDA主题模型建模，看看这些新闻主要在说哪些topic。

首先我们要把文本内容处理成固定的格式，一个包含句子的list，list中每个元素是分词后的词list。类似下面这个样子。

[[第，一，条，新闻，在，这里],[第，二，条，新闻，在，这里],[这，是，在，做， 什么],...]


In [10]:
from gensim import corpora, models, similarities
import gensim

### 载入停用词

In [11]:
stopwords=pd.read_csv("stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

### 转换成合适的格式

In [13]:
import jieba
import pandas as pd
df = pd.read_csv("technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()

sentences=[]
for line in lines:
    try:
        segs=jieba.lcut(line)
        segs = [x for x in segs if len(x)>1]
        segs = [x for x in segs if x not in stopwords]
        sentences.append(segs)
    except Exception as e:
        print(line)
        continue

In [14]:
for word in sentences[5]:
    print(word)

本次
商汤
带来
黄仁勋
展示
遥相呼应
SenseFace
人脸
布控
系统
千万级
人员
库中
300ms
识别
瞬间
锁定目标
功耗
十几
当属
人脸
布控
一大
科技


### 词袋模型

In [15]:
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [16]:
corpus[5]

[(21, 1),
 (39, 1),
 (61, 1),
 (68, 1),
 (78, 1),
 (82, 1),
 (91, 1),
 (92, 1),
 (103, 1),
 (104, 2),
 (105, 2),
 (124, 1),
 (129, 1),
 (130, 1),
 (131, 1),
 (132, 1),
 (133, 1),
 (134, 1),
 (135, 1),
 (136, 1),
 (137, 1),
 (138, 1)]

### LDA建模

In [17]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

我们查一下第3号分类，其中最常出现的单词是：

In [18]:
print(lda.print_topic(3, topn=5))

0.043*"电视" + 0.035*"乐视" + 0.017*"充电" + 0.015*"比特" + 0.014*"量子"


我们把所有的主题打印出来看看

In [19]:
for topic in lda.print_topics(num_topics=20, num_words=8):
    print(topic[1])

0.048*"手机" + 0.020*"产品" + 0.020*"联想" + 0.017*"汽车" + 0.015*"品牌" + 0.015*"市场" + 0.013*"中国" + 0.010*"360"
0.037*"公司" + 0.032*"中国" + 0.020*"科技" + 0.020*"市场" + 0.015*"业务" + 0.015*"创业" + 0.014*"投资" + 0.012*"企业"
0.020*"青年" + 0.018*"商家" + 0.018*"处理器" + 0.018*"糯米" + 0.018*"赋能" + 0.016*"营销" + 0.016*"供应链" + 0.015*"零售"
0.043*"电视" + 0.035*"乐视" + 0.017*"充电" + 0.015*"比特" + 0.014*"量子" + 0.013*"显示" + 0.011*"电子" + 0.011*"技术"
0.122*"游戏" + 0.026*"腾讯" + 0.018*"IP" + 0.018*"玩家" + 0.015*"金立" + 0.014*"赛事" + 0.014*"无人机" + 0.013*"动漫"
0.031*"网络安全" + 0.027*"攻击" + 0.023*"网络" + 0.021*"信息" + 0.020*"漏洞" + 0.018*"360" + 0.016*"诈骗" + 0.015*"手机"
0.030*"增长" + 0.022*"国美" + 0.015*"同比" + 0.014*"市场" + 0.013*"销售" + 0.013*"网秦" + 0.013*"消费者" + 0.012*"文件"
0.054*"数据" + 0.025*"服务" + 0.014*"企业" + 0.014*"提供" + 0.013*"平台" + 0.010*"互联网" + 0.010*"能力" + 0.010*"体系"
0.028*"用户" + 0.023*"互联网" + 0.018*"网络" + 0.015*"平台" + 0.014*"流量" + 0.010*"运营商" + 0.009*"服务" + 0.009*"模式"
0.058*"病毒" + 0.048*"勒索" + 0.027*"京东" + 0.025*"亿元" + 0.021*"收入" + 0.019*

我们可以对新加入的文本，进行简单主题分类：

- lda.get_document_topics(bow)